In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from glob import glob
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')
import re

In [2]:
know_train = [pd.read_csv(path, index_col=0) for path in sorted(glob('./_data/train/*.csv'))]
know_test = [pd.read_csv(path, index_col=0) for path in sorted(glob('./_data/test/*.csv'))]
submission = pd.read_csv('./_data/sample_submission.csv')

## 공백 `' '` 으로 되어있는 결측치를 np.nan으로 변경

In [3]:
for train, test in zip(know_train, know_test):
    for col in test.columns:
        train[col].replace(' ', np.nan, inplace=True)
        test[col].replace(' ', np.nan, inplace=True)

In [4]:
i=0
for train, test in zip(know_train, know_test):
    print(f'{2017+i} train 결측치\n {train.isna().sum()}')
    print(f'{2017+i} test 결측치\n: {test.isna().sum()}')
    i+=1

2017 train 결측치
 aq1_1          0
aq1_2        585
aq2_1          0
aq2_2        861
aq3_1          0
            ... 
bq40        1301
bq41_1      1332
bq41_2      8222
bq41_3      1316
knowcode       0
Length: 155, dtype: int64
2017 test 결측치
: aq1_1        0
aq1_2      598
aq2_1        0
aq2_2      862
aq3_1        0
          ... 
bq39_2       0
bq40      1333
bq41_1    1371
bq41_2    8191
bq41_3    1342
Length: 154, dtype: int64
2018 train 결측치
 cq1            0
cq2            0
cq3            0
cq4            0
cq5            0
            ... 
bq40           3
bq41_1      1270
bq41_2      1388
bq41_3      7824
knowcode       0
Length: 140, dtype: int64
2018 test 결측치
: cq1          0
cq2          0
cq3          0
cq4          0
cq5          0
          ... 
bq39      1331
bq40         0
bq41_1    1298
bq41_2    1423
bq41_3    7794
Length: 139, dtype: int64
2019 train 결측치
 sq1            0
sq2            0
sq3            0
sq4            0
sq5            0
            ... 
bq30      

### 결측치가 포함된 열 확인

In [5]:
include_na_col_train = []
include_na_col_test = []
for i in range(4):
    include_na_col_train.append(know_train[i].columns[know_train[i].isna().sum()>0].tolist())
    include_na_col_test.append(know_test[i].columns[know_test[i].isna().sum()>0].tolist())

### 2017~2020 결측치가 발생한 열의 개수 모두 다름

In [6]:
print('결측치가 포함된 열 수')
for i in range(4):
    print(f'{2017+i} train : {len(include_na_col_train[0])} 개')
    print(f'{2017+i} test : {len(include_na_col_test[0])} 개')

결측치가 포함된 열 수
2017 train : 59 개
2017 test : 60 개
2018 train : 59 개
2018 test : 60 개
2019 train : 59 개
2019 test : 60 개
2020 train : 59 개
2020 test : 60 개


In [7]:
print('결측치가 포함된 열 목록')
print('='*50)
for i in range(4):
    print(f'{2017+i} train \n {include_na_col_train[0]}')
    print('-'*50)
    print(f'{2017+i} test : {include_na_col_test[0]}')
    print('='*50)

결측치가 포함된 열 목록
2017 train 
 ['aq1_2', 'aq2_2', 'aq3_2', 'aq4_2', 'aq5_2', 'aq6_2', 'aq7_2', 'aq8_2', 'aq9_2', 'aq10_2', 'aq11_2', 'aq12_2', 'aq13_2', 'aq14_2', 'aq15_2', 'aq16_2', 'aq17_2', 'aq18_2', 'aq19_2', 'aq20_2', 'aq21_2', 'aq22_2', 'aq23_2', 'aq24_2', 'aq25_2', 'aq26_2', 'aq27_2', 'aq28_2', 'aq29_2', 'aq30_2', 'aq31_2', 'aq32_2', 'aq33_2', 'aq34_2', 'aq35_2', 'aq36_2', 'aq37_2', 'aq38_2', 'aq39_2', 'aq40_2', 'aq41_2', 'bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_1', 'bq5_2', 'bq12_2', 'bq12_3', 'bq12_4', 'bq19_1', 'bq31', 'bq32', 'bq33', 'bq34', 'bq38_1', 'bq40', 'bq41_1', 'bq41_2', 'bq41_3']
--------------------------------------------------
2017 test : ['aq1_2', 'aq2_2', 'aq3_2', 'aq4_2', 'aq5_2', 'aq6_2', 'aq7_2', 'aq8_2', 'aq9_2', 'aq10_2', 'aq11_2', 'aq12_2', 'aq13_2', 'aq14_2', 'aq15_2', 'aq16_2', 'aq17_2', 'aq18_2', 'aq19_2', 'aq20_2', 'aq21_2', 'aq22_2', 'aq23_2', 'aq24_2', 'aq25_2', 'aq26_2', 'aq27_2', 'aq28_2', 'aq29_2', 'aq30_2', 'aq31_2', 'aq32_2', 'aq33_2', 'aq34_2', 'aq35_2'

## 데이터 타입 확인
### 2017 train: float64 1개, int64 94개 object 60개

In [8]:
know_train[0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9486 entries, 0 to 9485
Columns: 155 entries, aq1_1 to knowcode
dtypes: float64(1), int64(94), object(60)
memory usage: 11.3+ MB


In [9]:
# bq23 : 기술적 변화요인으로 인한 업무 대체 비율
know_train[0].select_dtypes('float64')

,bq23
idx,
0,10.0
1,50.0
2,40.0
3,30.0
4,40.0
...,...
9481,10.0
9482,20.0
9483,0.0


In [10]:
know_train[0].select_dtypes('int64').columns

Index(['aq1_1', 'aq2_1', 'aq3_1', 'aq4_1', 'aq5_1', 'aq6_1', 'aq7_1', 'aq8_1',
       'aq9_1', 'aq10_1', 'aq11_1', 'aq12_1', 'aq13_1', 'aq14_1', 'aq15_1',
       'aq16_1', 'aq17_1', 'aq18_1', 'aq19_1', 'aq20_1', 'aq21_1', 'aq22_1',
       'aq23_1', 'aq24_1', 'aq25_1', 'aq26_1', 'aq27_1', 'aq28_1', 'aq29_1',
       'aq30_1', 'aq31_1', 'aq32_1', 'aq33_1', 'aq34_1', 'aq35_1', 'aq36_1',
       'aq37_1', 'aq38_1', 'aq39_1', 'aq40_1', 'aq41_1', 'bq1', 'bq2', 'bq3',
       'bq4', 'bq5', 'bq6', 'bq7', 'bq8_1', 'bq8_2', 'bq8_3', 'bq9', 'bq10',
       'bq11', 'bq12_1', 'bq12_5', 'bq13', 'bq14', 'bq15_1', 'bq15_2',
       'bq15_3', 'bq16', 'bq17', 'bq18_1', 'bq18_2', 'bq18_3', 'bq18_4',
       'bq18_5', 'bq18_6', 'bq18_7', 'bq19', 'bq20', 'bq21', 'bq22', 'bq24_1',
       'bq24_2', 'bq24_3', 'bq24_4', 'bq24_5', 'bq24_6', 'bq24_7', 'bq24_8',
       'bq25', 'bq26', 'bq27', 'bq28', 'bq29', 'bq35', 'bq36', 'bq37', 'bq38',
       'bq39_1', 'bq39_2', 'knowcode'],
      dtype='object')

In [11]:
know_train[0].select_dtypes('object').columns

Index(['aq1_2', 'aq2_2', 'aq3_2', 'aq4_2', 'aq5_2', 'aq6_2', 'aq7_2', 'aq8_2',
       'aq9_2', 'aq10_2', 'aq11_2', 'aq12_2', 'aq13_2', 'aq14_2', 'aq15_2',
       'aq16_2', 'aq17_2', 'aq18_2', 'aq19_2', 'aq20_2', 'aq21_2', 'aq22_2',
       'aq23_2', 'aq24_2', 'aq25_2', 'aq26_2', 'aq27_2', 'aq28_2', 'aq29_2',
       'aq30_2', 'aq31_2', 'aq32_2', 'aq33_2', 'aq34_2', 'aq35_2', 'aq36_2',
       'aq37_2', 'aq38_2', 'aq39_2', 'aq40_2', 'aq41_2', 'bq4_1a', 'bq4_1b',
       'bq4_1c', 'bq5_1', 'bq5_2', 'bq12_2', 'bq12_3', 'bq12_4', 'bq19_1',
       'bq30', 'bq31', 'bq32', 'bq33', 'bq34', 'bq38_1', 'bq40', 'bq41_1',
       'bq41_2', 'bq41_3'],
      dtype='object')

* aq는 모두 int64값이어야 함.
* bq는 float와 텍스트 데이터 제외하고 int64로 변경해야 함.

# 결측치 처리 (2017)

In [12]:
know_train[0].loc[know_train[0]['aq1_1']==1, 'aq1_2'].isna().sum()

585

* aq1_1에 1. 중요하지 않다라고 답했을 경우 aq1_2 문항을 건너뛰기 때문에 결측치 존재
## aq_1 문항을 1이라고 답한 행의 aq_2문항을 0으로 변경

In [13]:
import re
for col in know_train[0].columns:
    if re.findall('aq.*_1', col):
        na_col = re.sub('_1', '_2', col)
        know_train[0].loc[(know_train[0][col] == 1), na_col] = 0
        print(f'{na_col}열 결측치: {know_train[0][na_col].isna().sum()}')

aq1_2열 결측치: 0
aq2_2열 결측치: 0
aq3_2열 결측치: 0
aq4_2열 결측치: 0
aq5_2열 결측치: 0
aq6_2열 결측치: 0
aq7_2열 결측치: 0
aq8_2열 결측치: 0
aq9_2열 결측치: 0
aq10_2열 결측치: 0
aq11_2열 결측치: 4
aq12_2열 결측치: 0
aq13_2열 결측치: 0
aq14_2열 결측치: 3
aq15_2열 결측치: 0
aq16_2열 결측치: 0
aq17_2열 결측치: 0
aq18_2열 결측치: 0
aq19_2열 결측치: 0
aq20_2열 결측치: 0
aq21_2열 결측치: 0
aq22_2열 결측치: 0
aq23_2열 결측치: 0
aq24_2열 결측치: 0
aq25_2열 결측치: 0
aq26_2열 결측치: 0
aq27_2열 결측치: 7
aq28_2열 결측치: 0
aq29_2열 결측치: 0
aq30_2열 결측치: 0
aq31_2열 결측치: 0
aq32_2열 결측치: 11
aq33_2열 결측치: 0
aq34_2열 결측치: 0
aq35_2열 결측치: 0
aq36_2열 결측치: 0
aq37_2열 결측치: 0
aq38_2열 결측치: 0
aq39_2열 결측치: 0
aq40_2열 결측치: 0
aq41_2열 결측치: 4


In [14]:
for col in know_test[0].columns:
    if re.findall('aq.*_1', col):
        na_col = re.sub('_1', '_2', col)
        know_test[0].loc[(know_test[0][col] == 1), na_col] = 0
        print(f'{na_col}열 결측치: {know_test[0][na_col].isna().sum()}')

aq1_2열 결측치: 0
aq2_2열 결측치: 0
aq3_2열 결측치: 0
aq4_2열 결측치: 0
aq5_2열 결측치: 0
aq6_2열 결측치: 0
aq7_2열 결측치: 0
aq8_2열 결측치: 0
aq9_2열 결측치: 0
aq10_2열 결측치: 0
aq11_2열 결측치: 6
aq12_2열 결측치: 0
aq13_2열 결측치: 0
aq14_2열 결측치: 9
aq15_2열 결측치: 0
aq16_2열 결측치: 0
aq17_2열 결측치: 0
aq18_2열 결측치: 0
aq19_2열 결측치: 0
aq20_2열 결측치: 0
aq21_2열 결측치: 0
aq22_2열 결측치: 0
aq23_2열 결측치: 0
aq24_2열 결측치: 0
aq25_2열 결측치: 0
aq26_2열 결측치: 0
aq27_2열 결측치: 6
aq28_2열 결측치: 0
aq29_2열 결측치: 0
aq30_2열 결측치: 0
aq31_2열 결측치: 0
aq32_2열 결측치: 10
aq33_2열 결측치: 0
aq34_2열 결측치: 0
aq35_2열 결측치: 0
aq36_2열 결측치: 0
aq37_2열 결측치: 0
aq38_2열 결측치: 0
aq39_2열 결측치: 0
aq40_2열 결측치: 0
aq41_2열 결측치: 5


In [15]:
print(f'train\n{know_train[0].columns[know_train[0].isna().sum()>0].tolist()}')
print('-'*50)
print(f'test\n{know_test[0].columns[know_test[0].isna().sum()>0].tolist()}')

train
['aq11_2', 'aq14_2', 'aq27_2', 'aq32_2', 'aq41_2', 'bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_1', 'bq5_2', 'bq12_2', 'bq12_3', 'bq12_4', 'bq19_1', 'bq31', 'bq32', 'bq33', 'bq34', 'bq38_1', 'bq40', 'bq41_1', 'bq41_2', 'bq41_3']
--------------------------------------------------
test
['aq11_2', 'aq14_2', 'aq27_2', 'aq32_2', 'aq41_2', 'bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_1', 'bq5_2', 'bq12_2', 'bq12_3', 'bq12_4', 'bq19_1', 'bq30', 'bq31', 'bq32', 'bq33', 'bq34', 'bq38_1', 'bq40', 'bq41_1', 'bq41_2', 'bq41_3']


## 'aq11_2', 'aq14_2', 'aq27_2', 'aq32_2', 'aq41_2'문항에 결측치가 여전히 존재함.

In [16]:
know_train[0].loc[know_train[0]['aq11_2'].isna(), ['aq11_1','aq11_2']]

,aq11_1,aq11_2
idx,,
89,3,NaN
577,3,NaN
2422,3,NaN
4568,3,NaN


* 모두 aq11_1에 3. '중요하다' 라고 답했으나 aq11_2 항목을 건너뜀..'ㅅ';; aq11_1=3을 선택한 사람들이 aq11_2를 어떻게 답했는지 확인

In [17]:
know_train[0].loc[know_train[0]['aq11_1']==3, 'aq11_2'].value_counts()

4    1102
3     591
5     377
2      86
6      37
1       8
7       6
Name: aq11_2, dtype: int64

In [18]:
know_train[0].loc[(know_train[0]['aq11_1']==3)& (know_train[0]['aq11_2'].isna()==False), 'aq11_2'].astype(int).median()

4.0

In [19]:
know_train[0].loc[89, 'aq11_2'] = know_train[0].loc[(know_train[0]['aq11_1']==3) & (know_train[0]['aq11_2'].isna()==False), 'aq11_2'].astype(int).median()

중앙값으로 결측치를 채움

## 다른 열들도 동일한 방법으로 결측치 채우기

In [20]:
na_cols = ['aq11_2', 'aq14_2', 'aq27_2', 'aq32_2', 'aq41_2']
for col in na_cols:
    na_indexs = know_train[0][know_train[0][col].isna()].index.tolist()
    for index in na_indexs:
        forward_col = re.sub('_2', '_1', col)
        
        # _1에 한 응답
        val_1 = know_train[0].iloc[index][forward_col]
        
        # 같은 응답한 사람들의 _2 중앙값
        val_2 = know_train[0].loc[(know_train[0][forward_col]==val_1)& (know_train[0][col].isna()==False), col].astype(int).median()
        
        know_train[0].loc[index, col] = val_2
        print(f'{index}행 {col}열의 결측치를 {forward_col}에 같은 응답(={val_1})을 한 사람들의 중앙값 {val_2}으로 대체합니다.')

577행 aq11_2열의 결측치를 aq11_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체합니다.
2422행 aq11_2열의 결측치를 aq11_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체합니다.
4568행 aq11_2열의 결측치를 aq11_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체합니다.
3137행 aq14_2열의 결측치를 aq14_1에 같은 응답(=4)을 한 사람들의 중앙값 5.0으로 대체합니다.
4564행 aq14_2열의 결측치를 aq14_1에 같은 응답(=4)을 한 사람들의 중앙값 5.0으로 대체합니다.
6539행 aq14_2열의 결측치를 aq14_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체합니다.
715행 aq27_2열의 결측치를 aq27_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체합니다.
792행 aq27_2열의 결측치를 aq27_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체합니다.
3960행 aq27_2열의 결측치를 aq27_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체합니다.
4060행 aq27_2열의 결측치를 aq27_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체합니다.
5388행 aq27_2열의 결측치를 aq27_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체합니다.
6383행 aq27_2열의 결측치를 aq27_1에 같은 응답(=4)을 한 사람들의 중앙값 5.0으로 대체합니다.
9449행 aq27_2열의 결측치를 aq27_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체합니다.
1458행 aq32_2열의 결측치를 aq32_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체합니다.
2680행 aq32_2열의 결측치를 aq32_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체합니다.
3523행 aq32_2열의 결측치를 aq32_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체

In [22]:
na_cols = ['aq11_2', 'aq14_2', 'aq27_2', 'aq32_2', 'aq41_2']
for col in na_cols:
    na_indexs = know_test[0][know_test[0][col].isna()].index.tolist()
    for index in na_indexs:
        forward_col = re.sub('_2', '_1', col)
        
        # _1에 같은 응답
        val_1 = know_test[0].iloc[index][forward_col]
        
        # 같은 응답한 사람들의 _2 중앙값
        val_2 = know_test[0].loc[(know_test[0][forward_col]==val_1)& (know_test[0][col].isna()==False), col].astype(int).median()
        
        know_test[0].loc[index, col] = val_2
        print(f'{index}행 {col}열의 결측치를 {forward_col}에 같은 응답(={val_1})을 한 사람들의 중앙값 {val_2}으로 대체합니다.')

4319행 aq11_2열의 결측치를 aq11_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체합니다.
5290행 aq11_2열의 결측치를 aq11_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체합니다.
5960행 aq11_2열의 결측치를 aq11_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체합니다.
6775행 aq11_2열의 결측치를 aq11_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체합니다.
6877행 aq11_2열의 결측치를 aq11_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체합니다.
7350행 aq11_2열의 결측치를 aq11_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체합니다.
723행 aq14_2열의 결측치를 aq14_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체합니다.
1234행 aq14_2열의 결측치를 aq14_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체합니다.
1452행 aq14_2열의 결측치를 aq14_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체합니다.
2026행 aq14_2열의 결측치를 aq14_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체합니다.
3083행 aq14_2열의 결측치를 aq14_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체합니다.
5078행 aq14_2열의 결측치를 aq14_1에 같은 응답(=4)을 한 사람들의 중앙값 5.0으로 대체합니다.
6270행 aq14_2열의 결측치를 aq14_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체합니다.
7139행 aq14_2열의 결측치를 aq14_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체합니다.
7320행 aq14_2열의 결측치를 aq14_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 대체합니다.
2302행 aq27_2열의 결측치를 aq27_1에 같은 응답(=3)을 한 사람들의 중앙값 4.0으로 

## 7749행 aq32_2열의 결측치를 aq32_1에 같은 응답(=33)을 한 사람들의 중앙값 nan으로 대체합니다.
* 33..?

In [23]:
know_test[0].iloc[7749]['aq32_1']

33

In [24]:
know_test[0].loc[7749,'aq32_1'] = 3

In [25]:
know_test[0].loc[7749,'aq32_2'] = know_test[0].loc[(know_test[0]['aq32_1']==3) & (know_test[0]['aq32_2'].isna()==False), 'aq32_2'].astype(int).median()

In [26]:
know_test[0].loc[7749,'aq32_2']

4.0

# 이상치 확인
## 값의 범위
* aq*_1 : 1~5
* aq*_2 : 1~7

In [27]:
for col in know_train[0].columns:
    if re.findall('aq.*_1', col):
        outofval = know_train[0].loc[know_train[0][col].astype(int)>5,col].index.tolist()
        if outofval:
            print(f'{outofval}행 {col}열에 이상치 존재')
    elif re.findall('aq.*_2', col):
        outofval = know_train[0].loc[know_train[0][col].astype(int)>7,col].index.tolist()
        if outofval:
            print(f'{outofval}행 {col}열에 이상치 존재')

[3985]행 aq41_1열에 이상치 존재


In [28]:
know_train[0].loc[3985, 'aq41_1']

33

In [29]:
know_train[0].loc[3985, 'aq41_1'] = 3

In [30]:
for col in know_test[0].columns:
    if re.findall('aq.*_1', col):
        outofval = know_test[0].loc[know_test[0][col].astype(int)>5,col].index.tolist()
        if outofval:
            print(f'{outofval}행 {col}열에 이상치 존재')
    elif re.findall('aq.*_2', col):
        outofval = know_test[0].loc[know_test[0][col].astype(int)>7,col].index.tolist()
        if outofval:
            print(f'{outofval}행 {col}열에 이상치 존재')

* 나중에 다시 할때는 이상치 먼저 변경
* know_test[0].loc[7749,'aq32_1'] = 3
* know_train[0].loc[3985, 'aq41_1'] = 3


## 결측치 재확인

In [35]:
print(f'train\n{know_train[0].columns[know_train[0].isna().sum()>0].tolist()}')
print('-'*50)
print(f'test\n{know_test[0].columns[know_test[0].isna().sum()>0].tolist()}')

train
['bq4_1b', 'bq4_1c', 'bq5_1', 'bq5_2', 'bq12_2', 'bq12_3', 'bq12_4', 'bq19_1', 'bq31', 'bq32', 'bq33', 'bq34', 'bq38_1', 'bq40', 'bq41_1', 'bq41_2', 'bq41_3']
--------------------------------------------------
test
['bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_1', 'bq5_2', 'bq12_2', 'bq12_3', 'bq12_4', 'bq19_1', 'bq30', 'bq31', 'bq32', 'bq33', 'bq34', 'bq38_1', 'bq40', 'bq41_1', 'bq41_2', 'bq41_3']


# bq 문항 이상치 확인
* bq1 : 1~21
* bq2, bq3, bq22: 1~6
* bq6, bq7: 1~7
* bq8_1 ~ bq19, bq21, bq25 ~ 29: 1 ~ 5
* bq19_1 : 텍스트
* bq20 :1~4
* bq23 : 기술적 변화요인으로 인한 업무 대체 비율, float 1~100
* bq24_1 ~ 8 : 1 ~ 2
* bq30 ~ 34: 텍스트
* bq36: 성별 1.남 2.여
* bq37: 나이
* bq38: 학력
    * bq38_1: 학과 텍스트
* bq39_1: 고용형태, 1~4 선택시 bq40
* bq39_2: 5~7 선택시 bq41_3
* bq40: 1.정규직 2비정규직
* bq41_1~3 : 소득

# bq4
* bq4_1a : 업무에서 요구되는 자격증. bq4==1일때만, 텍스트 데이터
* bq4_1b : 업무에서 요구되는 자격증. bq4==1일때만, 텍스트 데이터
* bq4_1c : 업무에서 요구되는 자격증. bq4==1일때만, 텍스트 데이터
### bq4=2라고 대답할 경우 a,b,c 미응답

In [31]:
know_train[0][['bq4','bq4_1a','bq4_1b','bq4_1c']]

,bq4,bq4_1a,bq4_1b,bq4_1c
idx,,,,
0,1,자동차도장기능사,NaN,NaN
1,1,건축전기설비기술사,NaN,NaN
2,1,건축전기설비기술사,NaN,NaN
3,1,지적기사,NaN,NaN
4,1,건축전기설비기술사,NaN,NaN
...,...,...,...,...
9481,2,NaN,NaN,NaN
9482,2,NaN,NaN,NaN
9483,2,NaN,NaN,NaN


In [32]:
know_train[0].loc[know_train[0].bq4==2, ['bq4_1a','bq4_1b','bq4_1c']]= '없음'

In [33]:
know_train[0][['bq4','bq4_1a','bq4_1b','bq4_1c']]

,bq4,bq4_1a,bq4_1b,bq4_1c
idx,,,,
0,1,자동차도장기능사,NaN,NaN
1,1,건축전기설비기술사,NaN,NaN
2,1,건축전기설비기술사,NaN,NaN
3,1,지적기사,NaN,NaN
4,1,건축전기설비기술사,NaN,NaN
...,...,...,...,...
9481,2,없음,없음,없음
9482,2,없음,없음,없음
9483,2,없음,없음,없음


In [34]:
know_train[0].loc[know_train[0].bq4==1, ['bq4_1b','bq4_1c']].fillna('없음')

,bq4_1b,bq4_1c
idx,,
0,없음,없음
1,없음,없음
2,없음,없음
3,없음,없음
4,없음,없음
...,...,...
9470,없음,없음
9472,제빵기능사,없음
9473,변액보험판매관리사,없음


In [87]:
know_test[0][['bq4','bq4_1a','bq4_1b','bq4_1c']]

,bq4,bq4_1a,bq4_1b,bq4_1c
idx,,,,
0,2,NaN,NaN,NaN
1,1,품질관리기사,NaN,NaN
2,2,NaN,NaN,NaN
3,2,NaN,NaN,NaN
4,2,NaN,NaN,NaN
...,...,...,...,...
9481,1,굴삭기 운전,NaN,NaN
9482,2,NaN,NaN,NaN
9483,1,건설기계 면허,NaN,NaN


In [88]:
know_test[0].loc[know_test[0].bq4==2, ['bq4_1a','bq4_1b','bq4_1c']]= '없음'
know_test[0][['bq4','bq4_1a','bq4_1b','bq4_1c']]

,bq4,bq4_1a,bq4_1b,bq4_1c
idx,,,,
0,2,없음,없음,없음
1,1,품질관리기사,NaN,NaN
2,2,없음,없음,없음
3,2,없음,없음,없음
4,2,없음,없음,없음
...,...,...,...,...
9481,1,굴삭기 운전,NaN,NaN
9482,2,없음,없음,없음
9483,1,건설기계 면허,NaN,NaN


In [89]:
know_test[0].loc[know_test[0].bq4==1, ['bq4_1b','bq4_1c']].fillna('없음')

,bq4_1b,bq4_1c
idx,,
1,없음,없음
6,없음,없음
13,수질환경산업기사,없음
14,없음,없음
18,없음,없음
...,...,...
9479,없음,없음
9481,없음,없음
9483,없음,없음


In [92]:
know_train[0].loc[know_train[0].bq4==1, ['bq4_1a']].isna().sum()

bq4_1a    0
dtype: int64

In [91]:
know_test[0].loc[know_test[0].bq4==1, ['bq4_1a']].isna().sum()

bq4_1a    0
dtype: int64

# bq5
* bq5 : 1~2 훈련 필요하다/필요없다
    * bq5_1 : 1~6, 훈련기간, bq5==1일때만
    * bq5_2 : 훈련종류, bq5==1일때만, 텍스트

In [36]:
know_train[0][['bq5','bq5_1','bq5_2']]

,bq5,bq5_1,bq5_2
idx,,,
0,1,2,실무교육
1,2,NaN,NaN
2,2,NaN,NaN
3,1,1,측량 및 정보통신기술 교육
4,2,NaN,NaN
...,...,...,...
9481,1,2,편집기술에 원리와 이해능력
9482,1,5,"도면 작성 교육, 전자 기초 이해"
9483,2,NaN,NaN


In [39]:
know_train[0].loc[know_train[0].bq5==2,['bq5_1','bq5_2']] = '필요없음'

In [40]:
know_train[0].loc[know_train[0].bq5==2,['bq5_1','bq5_2']]

,bq5_1,bq5_2
idx,,
1,필요없음,필요없음
2,필요없음,필요없음
4,필요없음,필요없음
7,필요없음,필요없음
12,필요없음,필요없음
...,...,...
9478,필요없음,필요없음
9480,필요없음,필요없음
9483,필요없음,필요없음


In [42]:
know_train[0].loc[know_train[0].bq5==1,['bq5_1','bq5_2']].isna().sum()

bq5_1    0
bq5_2    1
dtype: int64

In [52]:
know_train[0].loc[know_train[0].bq5==1, 'bq5_2'] = '미응답'

In [81]:
know_test[0][['bq5','bq5_1','bq5_2']]

,bq5,bq5_1,bq5_2
idx,,,
0,2,NaN,NaN
1,1,1,실무교육
2,1,6,"기획력, 현장경험 등"
3,2,NaN,NaN
4,2,NaN,NaN
...,...,...,...
9481,2,NaN,NaN
9482,1,1,"도면의 이해,기본서류작성(PPT,EXCEL)"
9483,2,NaN,NaN


In [83]:
know_test[0].loc[know_test[0].bq5==2,['bq5_1','bq5_2']] = '필요없음'
know_test[0].loc[know_test[0].bq5==2,['bq5_1','bq5_2']]

,bq5_1,bq5_2
idx,,
0,필요없음,필요없음
3,필요없음,필요없음
4,필요없음,필요없음
6,필요없음,필요없음
9,필요없음,필요없음
...,...,...
9477,필요없음,필요없음
9478,필요없음,필요없음
9480,필요없음,필요없음


In [84]:
know_test[0].loc[know_test[0].bq5==1,['bq5_1','bq5_2']].isna().sum()

bq5_1    0
bq5_2    1
dtype: int64

In [85]:
know_test[0].loc[know_test[0].bq5==1, 'bq5_2'] = '미응답'

# bq12
* 'bq12_2', 'bq12_3', 'bq12_4',

In [71]:
know_train[0][['bq12_2','bq12_3','bq12_4']].isna().sum()

bq12_2    1630
bq12_3    1343
bq12_4     879
dtype: int64

In [ ]:
print(know_train[0][know_train[0].bq12_2==9]['bq12_2'])
print(know_train[0][know_train[0].bq12_3==9]['bq12_3'])
print(know_train[0][know_train[0].bq12_4==9]['bq12_4'])

* 'bq12_2','bq12_3','bq12_4' 열에 9값이 존재하지 않음. 따라서 na값을 9 해당없음으로 변경

In [73]:
know_train[0].loc[know_train[0]['bq12_2'].isna(), 'bq12_2']= 9
know_train[0].loc[know_train[0]['bq12_3'].isna(), 'bq12_3']= 9
know_train[0].loc[know_train[0]['bq12_4'].isna(), 'bq12_4']= 9

In [74]:
know_test[0][['bq12_2','bq12_3','bq12_4']].isna().sum()

bq12_2    1609
bq12_3    1332
bq12_4     884
dtype: int64

In [79]:
print(know_test[0][know_test[0].bq12_2==9]['bq12_2'])
print(know_test[0][know_test[0].bq12_3==9]['bq12_3'])
print(know_test[0][know_test[0].bq12_4==9]['bq12_4'])

Series([], Name: bq12_2, dtype: object)
Series([], Name: bq12_3, dtype: object)
Series([], Name: bq12_4, dtype: object)


In [80]:
know_test[0].loc[know_test[0]['bq12_2'].isna(), 'bq12_2']= 9
know_test[0].loc[know_test[0]['bq12_3'].isna(), 'bq12_3']= 9
know_test[0].loc[know_test[0]['bq12_4'].isna(), 'bq12_4']= 9

# 'bq19_1',
### bq19라고 생각하는 이유(일자리변화). 텍스트

In [95]:
know_train[0].loc[know_train[0]['bq19_1'].isna(),['bq19','bq19_1']]

,bq19,bq19_1
idx,,
227,2,NaN
811,3,NaN
1649,3,NaN
2986,3,NaN
4271,3,NaN
5673,2,NaN
8815,2,NaN


In [96]:
know_train[0].loc[know_train[0]['bq19_1'].isna(),'bq19_1'] = '이유없음'

In [94]:
know_test[0].loc[know_test[0]['bq19_1'].isna(),['bq19','bq19_1']]

,bq19,bq19_1
idx,,
201,3,NaN
2101,3,NaN
3287,4,NaN
4301,3,NaN
5371,3,NaN
7264,2,NaN
7640,3,NaN


In [97]:
know_test[0].loc[know_test[0]['bq19_1'].isna(),'bq19_1'] = '이유없음'

In [98]:
print(know_train[0]['bq19_1'].isna().sum())
print(know_test[0]['bq19_1'].isna().sum())

0
0


 # 'bq30' 
 ## 직업을 다르게 부르는 명칭이 있다면?

In [99]:
print(know_train[0]['bq30'].isna().sum())
print(know_test[0]['bq30'].isna().sum())

0
1


In [ ]:
know_test[0].loc[know_test[0]['bq30'].isna(),'bq30']

In [103]:
know_train[0]['bq30'].value_counts()

없다        6606
선생님        274
없음         251
기사          96
연구원         55
          ... 
갤러리스트        1
슈즈디자이너       1
팻턴사          1
도장 기사        1
음악제작자        1
Name: bq30, Length: 1199, dtype: int64

In [102]:
know_test[0]['bq30'].value_counts()

없다         6561
없음          292
선생님         274
기사           81
연구원          71
           ... 
기술사           1
소프트엔지니어       1
그림작가          1
공인회계사         1
3D자이너         1
Name: bq30, Length: 1181, dtype: int64

In [105]:
know_train[0].loc[know_train[0]['bq30']=='없다', 'bq30'] = '없음'
know_test[0].loc[know_test[0]['bq30']=='없다', 'bq30'] = '없음'
know_test[0].loc[know_test[0]['bq30'].isna(),'bq30'] = '없음'

In [106]:
print(know_train[0]['bq30'].isna().sum())
print(know_test[0]['bq30'].isna().sum())

0
0


# 'bq31', 
## 업무에 활용하는 도구나 프로그램

In [107]:
print(know_train[0]['bq31'].isna().sum())
print(know_test[0]['bq31'].isna().sum())

8
5


In [109]:
know_train[0]['bq31'].value_counts()

컴퓨터                                          511
없다                                           497
PC                                           229
컴퓨터, 엑셀                                       31
장갑                                            29
                                            ... 
문서 작성, 도면 작성 프로그램                              1
프리미어, 3Dmax, 포토샵                               1
피아노,악보                                         1
컴퓨터, NCS, 한글, MSoffice program, JAVA, 포토샵      1
PC, 공구, 설명서                                    1
Name: bq31, Length: 6777, dtype: int64

In [110]:
know_test[0]['bq31'].value_counts()

없다                   480
컴퓨터                  470
PC                   216
엑셀                    38
장갑                    34
                    ... 
성형기, 가마                1
PC,3DCAD, 그래픽프로그램      1
한도 시스템                 1
쇼핑카터, 정리도구             1
컴퓨터,혈압측정기,온도계          1
Name: bq31, Length: 6829, dtype: int64

In [112]:
know_train[0].loc[know_train[0].bq31.isna(),'bq31'] = '없다'
know_test[0].loc[know_test[0].bq31.isna(),'bq31'] = '없다'

# 'bq32'
## 직전직업

In [113]:
know_train[0]['bq32'].value_counts()

없다                 5723
없음                  210
주부                  147
회사원                 125
학생                  102
                   ... 
선박관련쪽(구체적으로 거절)       1
패션회사 알바               1
공작기계 개발연구원            1
보일러기사                 1
신발가게 운영               1
Name: bq32, Length: 1568, dtype: int64

In [114]:
know_test[0]['bq32'].value_counts()

없다                         5565
없음                          235
주부                          155
회사원                         128
학생                          120
                           ... 
같은 교사인데 다른 담당을 했음(국어교사)       1
영상관련 회사원                      1
기계, 영업업무                      1
판매 아르바이트                      1
총무 사무원                        1
Name: bq32, Length: 1620, dtype: int64

In [115]:
know_train[0].loc[know_train[0].bq32.isna(),'bq32'] = '없음'
know_test[0].loc[know_test[0].bq32.isna(),'bq32'] = '없음'
know_train[0].loc[know_train[0].bq32=='없다','bq32'] = '없음'
know_test[0].loc[know_test[0].bq32=='없다','bq32'] = '없음'

# 'bq33'
## 경력을 활용하여 전직 가능한 직업

In [116]:
know_train[0]['bq33'].value_counts()

없다                        6667
모름                         434
없음                         218
교수                          43
대학교수                        22
                          ... 
교수, 강사                       1
제품디자이너, 인테리어 디자이너, UMD       1
의료심사                         1
보건소장, 개인클리닉원장                1
피아노 학원원장                     1
Name: bq33, Length: 1418, dtype: int64

In [117]:
know_test[0]['bq33'].value_counts()

없다                    6561
모름                     413
없음                     241
교수                      57
대학교수                    36
                      ... 
공연마케터, 드라마출연, 영화감독       1
화장품 품질관리 직업              1
미술일일강좌                   1
시간강사,학습교제저자,교수           1
문화예술재단기획                 1
Name: bq33, Length: 1428, dtype: int64

In [118]:
know_train[0].loc[know_train[0].bq33.isna(),'bq33'] = '모름'
know_test[0].loc[know_test[0].bq33.isna(),'bq33'] = '모름'
know_train[0].loc[know_train[0].bq33=='없다','bq33'] = '없음'
know_test[0].loc[know_test[0].bq33=='없다','bq33'] = '없음'

# 'bq34'
## 하고 있는 일과 관련된 신직업/직무

In [119]:
know_train[0]['bq34'].value_counts()

없다                   7785
없음                    344
모름                     50
심리치료사                   4
쇼핑몰                     4
                     ... 
체대입시학원                  1
특수교사                    1
UPN엔지니어, 방화벽 엔지니어       1
타루점                     1
재생토너 판매업체               1
Name: bq34, Length: 339, dtype: int64

In [120]:
know_test[0]['bq34'].value_counts()

없다                   7751
없음                    339
모름                     58
심리상담사                   4
모르겠음                    4
                     ... 
애견관련 유투브 운영,블로거활동       1
귀금속 분석 가공기술자            1
직영카센타                   1
스포츠 엔터테인먼트 운영           1
피부미용사                   1
Name: bq34, Length: 345, dtype: int64

In [121]:
know_train[0].loc[know_train[0].bq34.isna(),'bq34'] = '모름'
know_test[0].loc[know_test[0].bq34.isna(),'bq34'] = '모름'
know_train[0].loc[know_train[0].bq34=='없다','bq34'] = '없음'
know_test[0].loc[know_test[0].bq34=='없다','bq34'] = '없음'
know_train[0].loc[know_train[0].bq34=='모르겠음','bq34'] = '모름'
know_test[0].loc[know_test[0].bq34=='모르겠음','bq34'] = '모름'

In [122]:
print(know_train[0]['bq34'].isna().sum())
print(know_test[0]['bq34'].isna().sum())

0
0


 # 'bq38_1', 
 ## 최종학력 학과

In [123]:
know_train[0]['bq38_1'].value_counts()

인문계       408
기계        387
경영학       346
기계공학      203
컴퓨터공학     161
         ... 
아동심리치료      1
재활의학        1
의료기기        1
자동화공학       1
연예방송        1
Name: bq38_1, Length: 1452, dtype: int64

In [124]:
know_test[0]['bq38_1'].value_counts()

인문계       435
기계        384
경영학       314
기계공학      177
컴퓨터공학     166
         ... 
패션마케팅학      1
관광과         1
청소년상담       1
문예창작학과      1
산업미술학       1
Name: bq38_1, Length: 1417, dtype: int64

In [125]:
print(know_train[0]['bq38_1'].isna().sum())
print(know_test[0]['bq38_1'].isna().sum())

271
278


In [126]:
know_train[0].loc[know_train[0].bq38_1.isna(),'bq38_1'] = '미응답'
know_test[0].loc[know_test[0].bq38_1.isna(),'bq38_1'] = '미응답'

In [127]:
print(know_train[0]['bq38_1'].isna().sum())
print(know_test[0]['bq38_1'].isna().sum())

0
0


# bq39 고용형태
### bq39_1에 답변했을 경우 41-3 null
### bq39_2에 답변했을 경우 bq40, bq40_1, bq40_2 null 이어야함.
### bq39_1은 1 ~ 4의 값을 bq39_2는 5 ~ 7의 값을 가짐

In [134]:
know_train[0].loc[:,['bq39_1','bq39_2','bq40', 'bq41_1', 'bq41_2', 'bq41_3']]

,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3
idx,,,,,,
0,1,1,1,4000,NaN,2200
1,1,1,1,NaN,NaN,2400
2,1,1,1,4000,NaN,2400
3,1,1,1,7000,NaN,3500
4,1,1,1,4000,NaN,2500
...,...,...,...,...,...,...
9481,1,1,1,5200,NaN,1800
9482,1,1,1,4000,NaN,3000
9483,1,4,2,2700,NaN,1500


In [139]:
know_train[0].loc[know_train[0].bq39_2<5,['bq39_1','bq39_2','bq40', 'bq41_1', 'bq41_2', 'bq41_3']]

,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3
idx,,,,,,
0,1,1,1,4000,NaN,2200
1,1,1,1,NaN,NaN,2400
2,1,1,1,4000,NaN,2400
3,1,1,1,7000,NaN,3500
4,1,1,1,4000,NaN,2500
...,...,...,...,...,...,...
9481,1,1,1,5200,NaN,1800
9482,1,1,1,4000,NaN,3000
9483,1,4,2,2700,NaN,1500


In [140]:
know_test[0].loc[know_test[0].bq39_2<5,['bq39_1','bq39_2','bq40', 'bq41_1', 'bq41_2', 'bq41_3']]

,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3
idx,,,,,,
0,1,1,1,3000,NaN,2300
1,1,1,1,5500,NaN,2500
2,1,1,1,4300,NaN,4000
3,1,1,1,4100,NaN,3000
4,1,1,1,2800,NaN,2000
...,...,...,...,...,...,...
9479,1,1,1,3200,NaN,2500
9480,1,1,1,3700,NaN,1800
9482,1,1,1,7000,NaN,2400


In [135]:
know_test[0].loc[:,['bq39_1','bq39_2','bq40', 'bq41_1', 'bq41_2', 'bq41_3']]

,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3
idx,,,,,,
0,1,1,1,3000,NaN,2300
1,1,1,1,5500,NaN,2500
2,1,1,1,4300,NaN,4000
3,1,1,1,4100,NaN,3000
4,1,1,1,2800,NaN,2000
...,...,...,...,...,...,...
9481,2,6,NaN,NaN,3400,NaN
9482,1,1,1,7000,NaN,2400
9483,2,6,NaN,NaN,4500,NaN


# 'bq40', 
## 1 정규직 2 비정규직

In [128]:
print(know_train[0]['bq40'].isna().sum())
print(know_test[0]['bq40'].isna().sum())

1301
1333


# 'bq41_1',
## 근로소득. bq39_1 임금근로자(1~4일경우만)

# 'bq41_2', 
## 초임임금. bq39_1 임금근로자(1~4일경우만)

# 'bq41_3'
## 순수입. bq39_2 비임금근로자 (5~7일경우)

# 데이터 타입 변경

In [37]:
know_train[0].select_dtypes('object').columns

Index(['aq1_2', 'aq2_2', 'aq3_2', 'aq4_2', 'aq5_2', 'aq6_2', 'aq7_2', 'aq8_2',
       'aq9_2', 'aq10_2', 'aq11_2', 'aq12_2', 'aq13_2', 'aq14_2', 'aq15_2',
       'aq16_2', 'aq17_2', 'aq18_2', 'aq19_2', 'aq20_2', 'aq21_2', 'aq22_2',
       'aq23_2', 'aq24_2', 'aq25_2', 'aq26_2', 'aq27_2', 'aq28_2', 'aq29_2',
       'aq30_2', 'aq31_2', 'aq32_2', 'aq33_2', 'aq34_2', 'aq35_2', 'aq36_2',
       'aq37_2', 'aq38_2', 'aq39_2', 'aq40_2', 'aq41_2', 'bq4_1a', 'bq4_1b',
       'bq4_1c', 'bq5_1', 'bq5_2', 'bq12_2', 'bq12_3', 'bq12_4', 'bq19_1',
       'bq30', 'bq31', 'bq32', 'bq33', 'bq34', 'bq38_1', 'bq40', 'bq41_1',
       'bq41_2', 'bq41_3'],
      dtype='object')

In [43]:
# 텍스트 데이터
text_cols = ['bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq19_1', 'bq30', 'bq31', 'bq32', 'bq33', 'bq34', 'bq38_1']
type_dict ={col:'int64'  for col in know_train[0].select_dtypes('object').columns.tolist() if col not in text_cols}
know_train[0].astype(type_dict)
#nan값 있으면 안됨..ㅠㅠ

ValueError: cannot convert float NaN to integer

In [ ]:
# c = know_train[0].select_dtypes('int64').columns[:-1]
# fig, ax = plt.subplots(nrows=len(c)//3, ncols=3, figsize=(30, 100))

# for i, col_name in enumerate(c):
#     sns.countplot(x=col_name, data=know_train[0], ax=ax[i//3][i%3], palette='Set2')
# plt.show()

In [125]:
know_train[0].columns[know_train[0].isna().sum()>0]

Index(['bq4_1b', 'bq4_1c', 'bq5_1', 'bq5_2', 'bq12_2', 'bq12_3', 'bq12_4',
       'bq19_1', 'bq31', 'bq32', 'bq33', 'bq34', 'bq38_1', 'bq40', 'bq41_1',
       'bq41_2', 'bq41_3'],
      dtype='object')

In [128]:
know_train[0]['bq4_1a'].isna().sum()

0

In [ ]:
know_train[0]['bq4_1a']

In [80]:
know_train[0].loc[know_train[0]['aq14_2'].isna(), ['aq14_1','aq14_2']]

,aq14_1,aq14_2
idx,,
3137,4,NaN
4564,4,NaN
6539,3,NaN


In [81]:
know_train[0].loc[know_train[0]['aq32_2'].isna(), ['aq32_1','aq32_2']]

,aq27_1,aq27_2
idx,,
715,3,NaN
792,3,NaN
3960,3,NaN
4060,3,NaN
5388,3,NaN
6383,4,NaN
9449,3,NaN


In [82]:
know_train[0].loc[know_train[0]['aq41_2'].isna(), ['aq41_1','aq41_2']]

,aq41_1,aq41_2
idx,,
1798,3,NaN
6894,3,NaN
7118,3,NaN
8289,3,NaN
